In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from utils import processing
from utils import utils

from tensorflow.keras.models import Sequential
from tensorflow.keras import Input
from tensorflow.keras.layers import LSTM, GRU, Conv1D, MaxPooling1D, Flatten, Dense, Dropout, TimeDistributed, \
    BatchNormalization
from tensorflow.keras.optimizers import RMSprop, Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

import wandb
from wandb.keras import WandbCallback


In [2]:
plt.style.use('fivethirtyeight')
plt.rcParams["figure.figsize"] = (18,10)

In [3]:
wandb.login()

wandb: Currently logged in as: nbvanting (use `wandb login --relogin` to force relogin)


True

In [4]:
def load_data(config, dataset_name):
    # Load csv & parse dates to datetime index
    data = pd.read_csv(f'../data/processed/{dataset_name}_features.csv', index_col='Datetime', parse_dates=['Datetime'])
    # Select Features
    data = data[['Value', 'sunshine_mins', 'airtemp_c', 'daylength_hrs', 'wkdy_sin', 'wkdy_cos', 'wknd', 'mnth_sin', 'mnth_cos']]
    
    train, val, test = processing.create_datasets(data, split=split, 
                                                  steps=steps, lookback=config.lookback, 
                                                  horizon=horizon, batch_size=config.batch_size, 
                                                  scaler='standard')    
    return train, val, test
    
    
def build_model(config):
        
    model = Sequential()

    model.add(Input(shape=(config.lookback, config.num_features)))

    # CNN Block
    model.add(Conv1D(filters=config.cnn_layer_size_1, kernel_size=3, activation=config.activation_cnn))
    model.add(MaxPooling1D(pool_size=2))    
    for i in range(config.num_cnn_layers):
        model.add(Conv1D(filters=config.cnn_layer_size_2, kernel_size=3, activation=config.activation_cnn))
        model.add(MaxPooling1D(pool_size=2))
    
    # RNN Block
    for i in range(config.num_gru_layers):
        model.add(GRU(config.gru_layer_size_1, return_sequences=True, activation=config.activation_gru))
        model.add(Dropout(config.dropout))
    
    model.add(GRU(config.gru_layer_size_2, return_sequences=False, activation=config.activation_gru))
    model.add(Dropout(config.dropout))

    model.add(Dense(1))
    
    opt = config.optimizer
    if opt == 'sgd':
        opt = SGD(learning_rate=config.learning_rate, momentum=config.momentum)
    elif opt == 'rmsprop':
        opt = RMSprop(learning_rate=config.learning_rate)
    else:
        opt = Adam(learning_rate=config.learning_rate)
        
    model.compile(optimizer=opt, loss='mse', metrics=['mae', 'mape'])
    
    return model

def log_defaults():

    wandb_config = {
        'num_features' : 9,
        'epochs' : 50,
        'batch_size' : 32,
        'num_cnn_layers' : 1,
        'num_gru_layers' : 1,
        'optimizer' : 'adam',
        'dropout' : 0.5,
        'lookback' : 24*30,
        'activation_cnn' : 'relu',
        'activation_gru' : 'tanh',
        'cnn_layer_size_1' : 64,
        'cnn_layer_size_2' : 64,
        'gru_layer_size_1' : 50,
        'gru_layer_size_2' : 100,
        'learning_rate' : 0.001,
        'momentum' : 0.9,
    }
    return wandb_config
    

def run_tuner():
    
    wandb.init(config=log_defaults(), project='thesis')
    
    model = build_model(config=wandb.config)
    
    train, val, _ = load_data(config=wandb.config, dataset_name='kolding')

    callbacks = [WandbCallback()]
    
    model.fit(
        train,
        epochs=wandb.config.epochs,
        validation_data=val,
        callbacks=callbacks
    )

In [5]:
# Sweep Config
sweep_config = {
    'method': 'random',
    'metric': {
        'name': 'val_loss',
        'goal': 'minimize'
    },
    'early_terminate': {
        'type': 'hyperband',
        'min_iter': 5
    },
    'parameters': {
        'num_features' : {
            'value' : 9
        },
        'batch_size': {
            'distribution': 'q_log_uniform',
            'q': 1,
            'min': math.log(32),
            'max': math.log(256)
        },
        'lookback': {
            'distribution': 'int_uniform',
            'min': 24*7,
            'max': 24*31
        },
        'optimizer': {
            'values': ['rmsprop', 'sgd', 'adam']
        },
        'dropout': {
            'distribution': 'uniform',
            'min': 0,
            'max': 0.5
        },
        'epochs': {
            'value': 50
        },
        'activation_gru': {
            'value': 'tanh'
        },
        'activation_cnn': {
            'value': 'relu'
        },
        'cnn_layer_size_1': {
            'distribution': 'q_log_uniform',
            'q': 1,
            'min': math.log(32),
            'max': math.log(256)
        },
        'cnn_layer_size_2': {
            'distribution': 'q_log_uniform',
            'q': 1,
            'min': math.log(32),
            'max': math.log(256)
        },
        'gru_layer_size_1': {
            'distribution': 'q_log_uniform',
            'q': 1,
            'min': math.log(32),
            'max': math.log(256)
        },
        'gru_layer_size_2': {
            'distribution': 'q_log_uniform',
            'q': 1,
            'min': math.log(32),
            'max': math.log(256)
        },
        'learning_rate': {
            'distribution': 'uniform',
            'min': 0.001,
            'max': 0.1
        },
        'momentum': {
            'values': [0.8, 0.85, 0.9, 0.95]
        },
        'num_cnn_layers': {
            'values': [1, 2]
        },
        'num_gru_layers': {
            'values': [1, 2]
        }
    }
}

In [6]:
# Additional Parameters
split = 0.80 # split percentage for training data
steps = 1 # timesteps: 1 hour
horizon = 1 # the target hour in the future we want to predict 1 hour ahead


In [7]:
sweep_id = wandb.sweep(sweep_config, project='thesis')

Create sweep with ID: y7xar0dh
Sweep URL: https://wandb.ai/nbvanting/thesis/sweeps/y7xar0dh


In [ ]:
wandb.agent(sweep_id, function=run_tuner, count=30)

wandb: Agent Starting Run: 2ckvdj53 with config:
wandb: 	activation_cnn: relu
wandb: 	activation_gru: tanh
wandb: 	batch_size: 130
wandb: 	cnn_layer_size_1: 57
wandb: 	cnn_layer_size_2: 51
wandb: 	dropout: 0.48513791567026154
wandb: 	epochs: 50
wandb: 	gru_layer_size_1: 118
wandb: 	gru_layer_size_2: 34
wandb: 	learning_rate: 0.06795239772836459
wandb: 	lookback: 555
wandb: 	momentum: 0.8
wandb: 	num_cnn_layers: 2
wandb: 	num_features: 9
wandb: 	num_gru_layers: 1
wandb: 	optimizer: adam
wandb: WARNING Ignored wandb.init() arg project when running a sweep
c:\users\nicol\envs\thesis\lib\site-packages\IPython\html.py:12: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  warn("The `IPython.html` package has been deprecated since IPython 4.0. "


Epoch 1/50
114/114 [==============================] - 32s 247ms/step - loss: 2.5134 - mae: 1.1315 - mape: 568.5951 - val_loss: 1.5980 - val_mae: 0.9060 - val_mape: 116.7106
Epoch 2/50
114/114 [==============================] - 25s 219ms/step - loss: 1.0005 - mae: 0.7731 - mape: 188.4544 - val_loss: 2.0299 - val_mae: 0.9914 - val_mape: 300.4307
Epoch 3/50
114/114 [==============================] - 25s 220ms/step - loss: 1.0240 - mae: 0.7797 - mape: 181.6530 - val_loss: 1.3756 - val_mae: 0.8906 - val_mape: 246.9705
Epoch 4/50
114/114 [==============================] - 23s 202ms/step - loss: 1.0133 - mae: 0.7850 - mape: 189.7667 - val_loss: 1.3932 - val_mae: 0.8892 - val_mape: 223.5679
Epoch 5/50
114/114 [==============================] - 23s 202ms/step - loss: 1.0276 - mae: 0.7863 - mape: 194.8492 - val_loss: 1.6111 - val_mae: 0.9079 - val_mape: 122.7724
Epoch 6/50
114/114 [==============================] - 23s 202ms/step - loss: 1.0447 - mae: 0.7908 - mape: 226.4052 - val_loss: 1.6035 -

114/114 [==============================] - 24s 206ms/step - loss: 1.0339 - mae: 0.7868 - mape: 194.2294 - val_loss: 1.4163 - val_mae: 0.8889 - val_mape: 197.8769
Epoch 49/50
114/114 [==============================] - 24s 207ms/step - loss: 1.0449 - mae: 0.7957 - mape: 176.6646 - val_loss: 1.3949 - val_mae: 0.8891 - val_mape: 221.5493
Epoch 50/50
114/114 [==============================] - 25s 222ms/step - loss: 1.0446 - mae: 0.7965 - mape: 226.5836 - val_loss: 1.6678 - val_mae: 0.9172 - val_mape: 149.4739


epoch,49
loss,1.06072
mae,0.80051
mape,200.51965
val_loss,1.6678
val_mae,0.91716
val_mape,149.47392
_runtime,1199
_timestamp,1620040160
_step,49
best_val_loss,1.35098


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,█▁▁▁▂▁▁▁▁▂▁▁▁▁▁▁▁▂▂▁▁▁▂▁▁▁▁▂▂▂▁▁▂▁▁▁▁▁▁▁
mae,█▁▁▁▂▁▂▁▂▂▁▂▂▂▂▂▂▂▃▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▂▂▁▂▂
mape,█▂▁▁▃▂▁▂▂▃▂▂▃▂▂▂▂▄▄▃▃▂▃▃▃▂▂▄▄▅▃▁▃▁▂▁▃▁▃▂
val_loss,▃█▁▁▃▄▃▃▃▁▂▃▅▄▁▃▃▇▇▂▅▆▂▃▇▆▄▃▁▂▃▁▂█▄▃▃▄▂▄
val_mae,▂▇▁▁▂▂▁▂▂▁▁▂▄▃▁▂▁▆▇▁▄▅▁▂▆▅▂▂▁▁▂▁▁█▂▂▂▃▁▃
val_mape,▁▇▆▅▂▂▂▁▁▇▃▁▄▂▅▁▁▇▇▂▄▆▄▁▆▅▂▁▇▃▁▅▄█▂▁▁▃▄▃
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███


...

Epoch 1/50
66/66 [==============================] - 23s 277ms/step - loss: 24.9860 - mae: 3.6159 - mape: 21401.7243 - val_loss: 6.7944 - val_mae: 2.3980 - val_mape: 2438.9023
Epoch 2/50
66/66 [==============================] - 18s 266ms/step - loss: 12.1038 - mae: 3.1105 - mape: 17252.5471 - val_loss: 9.3897 - val_mae: 2.8622 - val_mape: 2868.5659
Epoch 3/50
66/66 [==============================] - 18s 267ms/step - loss: 7.2454 - mae: 2.3554 - mape: 12288.3195 - val_loss: 13.7975 - val_mae: 3.5429 - val_mape: 3464.0093
Epoch 4/50
66/66 [==============================] - 18s 267ms/step - loss: 5.9873 - mae: 2.0312 - mape: 13521.2256 - val_loss: 2.2556 - val_mae: 1.3417 - val_mape: 1282.1586
Epoch 5/50
66/66 [==============================] - 18s 265ms/step - loss: 4.4496 - mae: 1.7736 - mape: 10073.5473 - val_loss: 2.9376 - val_mae: 1.5410 - val_mape: 1533.9121
Epoch 6/50
66/66 [==============================] - 18s 265ms/step - loss: 2.8191 - mae: 1.3749 - mape: 5826.5409 - val_loss: 1

epoch,7
loss,2.53077
mae,1.26404
mape,7655.06348
val_loss,1.65606
val_mae,1.13411
val_mape,974.56293
_runtime,151
_timestamp,1620065029
_step,7
best_val_loss,1.58553


epoch,▁▂▃▄▅▆▇█
loss,█▅▃▂▂▁▁▁
mae,█▇▄▃▂▁▁▁
mape,█▆▆▆▂▁▁▄
val_loss,▄▅█▁▂▁▆▁
val_mae,▅▆█▂▂▁▆▁
val_mape,▅▆█▂▃▁▆▁
_runtime,▁▂▃▄▅▆▇█
_timestamp,▁▂▃▄▅▆▇█
_step,▁▂▃▄▅▆▇█


wandb: Agent Starting Run: r7yneo68 with config:
wandb: 	activation_cnn: relu
wandb: 	activation_gru: tanh
wandb: 	batch_size: 38
wandb: 	cnn_layer_size_1: 225
wandb: 	cnn_layer_size_2: 128
wandb: 	dropout: 0.31963665946425573
wandb: 	epochs: 50
wandb: 	gru_layer_size_1: 104
wandb: 	gru_layer_size_2: 238
wandb: 	learning_rate: 0.053036436782389416
wandb: 	lookback: 470
wandb: 	momentum: 0.8
wandb: 	num_cnn_layers: 1
wandb: 	num_features: 9
wandb: 	num_gru_layers: 2
wandb: 	optimizer: adam
wandb: WARNING Ignored wandb.init() arg project when running a sweep


Epoch 1/50
391/391 [==============================] - 162s 399ms/step - loss: 4.8730 - mae: 1.4642 - mape: 2630.2871 - val_loss: 1.5800 - val_mae: 0.9033 - val_mape: 111.8551
Epoch 2/50
391/391 [==============================] - 163s 417ms/step - loss: 1.4660 - mae: 0.9458 - mape: 595.1624 - val_loss: 2.1494 - val_mae: 1.0332 - val_mape: 255.4025
Epoch 3/50
148/391 [==========>...................] - ETA: 1:35 - loss: 1.5886 - mae: 0.9867 - mape: 540.6973

In [ ]:
wandb.finish()